## Daskで時間のかかる前処理の高速化

最近仕事で自然言語処理を使った異常検知システムを作ってました。

異常検知といえば、Positive:Negativeの比率が1:10000とかそれ以上になってしまうため、
いくら訓練デーはアンダーサンプリング/オーバーサンプリング* をしたところで
実際の検知率や精度を検証するためにはかなりのデータ量が必要になります。

数百万件に対する前処理を

この記事は [PyLadies Advent Calendar 2018 - Adventar](https://adventar.org/calendars/3116) の18日目の記事になります。


基本的なDaskの使い方は
[Python Dask で 並列 DataFrame 処理 - StatsFragments](http://sinhrks.hatenablog.com/entry/2015/09/24/222735)
を見てみると良い

Daskのpandasインタフェースを使うと

複数のカラムの情報を用いて前処理を行うときのやり方

### 基本的な使い方

In [2]:
import pandas as pd
import dask

In [45]:
df = pd.read_csv('input/train.csv',)
print(f"{df.shape[0]:,d}")

159,571


In [ ]:
applyするだけ -> そのままでいい
countvecとかして別の形になる

In [49]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [52]:
import dask.dataframe as dd
import nltk
is_noun = lambda pos: pos[:2] == 'NN'

In [81]:
def get_nouns(sentence):
    tokenized = nltk.word_tokenize(sentence)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
    return nouns

In [72]:
%%time
df['comment_text'].head(100).apply(get_nouns)

CPU times: user 320 ms, sys: 5.47 ms, total: 326 ms
Wall time: 327 ms


0     [Explanation, edits, Hardcore, Metallica, Fan,...
1           [D'aww, colour, Thanks, talk, January, UTC]
2     [Hey, man, war, guy, information, edits, talk,...
3     [suggestions, improvement, section, statistics...
4                                  [hero, chance, page]
5                        [Congratulations, tools, talk]
6     [COCKSUCKER, BEFORE, YOU, PISS, AROUND, ON, MY...
7       [vandalism, Matt, Shirvington, article, Please]
8     [Sorry, word, 'nonsense, Anyway, anything, art...
9                       [alignment, subject, DuLithgow]
10    [Fair, rationale, Image, Thanks, Image, Wonju....
11                                    [bbq, man, phone]
12    [Hey, @, |, talk, group, WP, TALIBANS, purist,...
13    [accusations, warnings, edit, ad, hominem, att...
14    [girl, arguments, argument, Yvesnimmo, situati...
15    [Juelz, Santanas, Age, Juelz, Santana, years, ...
16                                        [Bye, Tosser]
17        [Talk, Voydan, Pop, Georgiev-, Chernod

In [68]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cocodrips/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [74]:
ddf = dd.from_pandas(df, 10)
ddf._meta.dtypes

id               object
comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object

In [80]:
res = ddf['comment_text'].map_partitions(get_nouns, axis=1, meta='list')
res.compute(scheduler='processes')

TypeError: get_nouns() got an unexpected keyword argument 'axis'

148万件のデータ。

In [13]:
df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [ ]:

脚注
https://qiita.com/ryouta0506/items/619d9ac0d80f8c0aed92